# σ Parameter Sensitivity Analysis for RBF Similarity

This notebook analyzes how the **bandwidth parameter σ** in the RBF kernel affects:
1. Local similarity structure within patches
2. Classification accuracy
3. Edge-case pixel performance

## RBF Kernel Formula

$$S(x_i, x_j) = \exp\left(-\frac{\|x_i - x_j\|^2}{2\sigma^2}\right)$$

- **Small σ**: Only very similar pixels have high similarity (local focus)
- **Large σ**: Similarity decays slowly (global smoothing)

## 1. Setup

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm

# Import our modules
from src.data import preprocess_pipeline
from src.features import extract_patches, split_dataset
from src.models import (
    rbf_similarity, local_similarity_matrix, compute_local_similarity_features,
    SVMClassifier, compute_all_metrics
)

# MLflow for tracking
import mlflow
mlflow.set_tracking_uri('mlruns')
mlflow.set_experiment('sigma_sensitivity_analysis')

# Set random seed for reproducibility
np.random.seed(42)

print("✓ Setup complete!")

## 2. Load or Generate Data

Uncomment the appropriate section based on whether you have downloaded the benchmark datasets.

In [ ]:
USE_REAL_DATA = False  # Set to True if you have downloaded Indian Pines

if USE_REAL_DATA:
    # Load Indian Pines dataset
    from src.data import load_benchmark_dataset
    cube = load_benchmark_dataset('indian_pines', 'data/external/indian_pines')
    hsi_data = cube.data
    ground_truth = cube.ground_truth
    class_names = cube.class_names
    print(f"Loaded Indian Pines: {hsi_data.shape}")
else:
    # Generate realistic synthetic data with spectral structure
    print("Generating synthetic HSI data with spectral structure...")
    
    height, width, n_bands = 100, 100, 200
    n_classes = 6
    
    # Create class-specific spectral signatures (more realistic)
    # Each class has a base spectrum + class-specific variations
    wavelengths = np.linspace(400, 2500, n_bands)
    
    # Base spectral patterns
    class_spectra = np.zeros((n_classes, n_bands))
    for c in range(n_classes):
        # Each class has peaks at different wavelengths
        peak_pos = 500 + c * 300  # Vary peak position by class
        class_spectra[c] = np.exp(-((wavelengths - peak_pos) ** 2) / (200 ** 2))
        # Add some secondary features
        class_spectra[c] += 0.3 * np.exp(-((wavelengths - (1500 + c * 100)) ** 2) / (100 ** 2))
    
    # Generate ground truth with spatial structure (clustered classes)
    ground_truth = np.zeros((height, width), dtype=np.int32)
    
    # Create regions for each class
    for c in range(1, n_classes):
        # Create random blobs for each class
        n_blobs = np.random.randint(3, 8)
        for _ in range(n_blobs):
            center_r = np.random.randint(5, height - 5)
            center_c = np.random.randint(5, width - 5)
            radius = np.random.randint(3, 10)
            
            for r in range(max(0, center_r - radius), min(height, center_r + radius)):
                for col in range(max(0, center_c - radius), min(width, center_c + radius)):
                    if ((r - center_r) ** 2 + (col - center_c) ** 2) < radius ** 2:
                        ground_truth[r, col] = c
    
    # Generate HSI cube based on ground truth with noise
    hsi_data = np.zeros((height, width, n_bands), dtype=np.float32)
    
    for c in range(n_classes):
        mask = ground_truth == c
        n_pixels = np.sum(mask)
        if n_pixels > 0:
            # Add spectral signature with intra-class variation
            base = class_spectra[c if c > 0 else 1]  # Use class 1 for background
            noise = 0.1 * np.random.randn(n_pixels, n_bands)
            hsi_data[mask] = base + noise
    
    class_names = ['Background'] + [f'Class_{c}' for c in range(1, n_classes)]
    print(f"Generated synthetic data: {hsi_data.shape}")

print(f"Ground truth classes: {np.unique(ground_truth)}")
print(f"Labeled samples: {np.sum(ground_truth > 0)}")

## 3. Preprocessing

In [ ]:
# Preprocess the data
result = preprocess_pipeline(
    hsi_data,
    remove_water=False,  # Already corrected for benchmark data
    reduce_dims='pca',
    n_components=30,     # Reduce to 30 PCA components
    normalize='minmax',
    log_to_mlflow=False
)

processed_data = result['data']
print(f"Preprocessed shape: {processed_data.shape}")

## 4. Analyze Sigma's Effect on Local Similarity

Visualize how different σ values affect the similarity structure within a patch.

In [ ]:
# Extract a sample patch
window_size = 7
half = window_size // 2

# Find a labeled pixel near class boundaries (interesting case)
labeled_positions = np.argwhere(ground_truth > 0)
sample_pos = labeled_positions[len(labeled_positions) // 2]
r, c = sample_pos

# Extract patch
padded = np.pad(processed_data, [(half, half), (half, half), (0, 0)], mode='reflect')
sample_patch = padded[r:r + window_size, c:c + window_size, :]

print(f"Sample patch from position ({r}, {c})")
print(f"Patch shape: {sample_patch.shape}")
print(f"Center pixel class: {ground_truth[r, c]}")

In [ ]:
# Analyze similarity matrix for different sigma values
sigma_values = [0.01, 0.05, 0.1, 0.25, 0.5, 1.0, 2.0, 5.0]

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i, sigma in enumerate(sigma_values):
    sim_matrix = local_similarity_matrix(sample_patch, metric='rbf', sigma=sigma)
    
    im = axes[i].imshow(sim_matrix, cmap='hot', vmin=0, vmax=1)
    axes[i].set_title(f'σ = {sigma}\nmean={np.mean(sim_matrix):.3f}')
    axes[i].axis('off')

plt.suptitle('RBF Similarity Matrix vs σ (Local Patch)', fontsize=14)
plt.tight_layout()

# Add colorbar
fig.colorbar(im, ax=axes, shrink=0.8, label='Similarity')
plt.savefig('sigma_similarity_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nObservation: Small σ creates sparse similarity (only very close spectra match),")
print("while large σ creates dense similarity (everything looks similar).")

## 5. σ Sensitivity on Classification Accuracy

Systematically test how σ affects SVM classification with RBF kernel.

In [ ]:
# Extract patches for classification
patch_size = 5
dataset = extract_patches(
    processed_data,
    ground_truth,
    window_size=patch_size,
    include_background=False,
    log_to_mlflow=False
)

# Split dataset
train_set, val_set, test_set = split_dataset(
    dataset,
    train_ratio=0.2,  # Use 20% for training (realistic for HSI)
    val_ratio=0.1,
    test_ratio=0.7,
    stratify=True,
    log_to_mlflow=False
)

# Get center pixels for SVM
X_train = train_set.get_center_pixels()
y_train = train_set.labels
X_test = test_set.get_center_pixels()
y_test = test_set.labels

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

In [ ]:
# Run sensitivity analysis
sigma_test_values = [0.001, 0.01, 0.05, 0.1, 0.25, 0.5, 1.0, 2.0, 5.0, 10.0]

results = []

print("Running σ sensitivity analysis...")
print("-" * 60)

for sigma in tqdm(sigma_test_values, desc="Testing σ values"):
    with mlflow.start_run(run_name=f'SVM_sigma_{sigma}'):
        # Log parameters
        mlflow.log_params({
            'sigma': sigma,
            'kernel': 'rbf',
            'C': 10.0,
            'patch_size': patch_size,
            'train_samples': len(X_train),
            'test_samples': len(X_test)
        })
        
        # Train SVM with specific gamma (gamma = 1 / (2 * sigma^2))
        gamma = 1.0 / (2 * sigma ** 2)
        
        svm = SVMClassifier(kernel='rbf', C=10.0, gamma=gamma)
        svm.fit(X_train, y_train, log_to_mlflow=False)
        
        # Evaluate
        y_pred = svm.predict(X_test)
        metrics = compute_all_metrics(y_test, y_pred)
        
        # Log metrics
        mlflow.log_metrics({
            'overall_accuracy': metrics['overall_accuracy'],
            'average_accuracy': metrics['average_accuracy'],
            'kappa': metrics['kappa_coefficient']
        })
        
        results.append({
            'sigma': sigma,
            'gamma': gamma,
            'OA': metrics['overall_accuracy'],
            'AA': metrics['average_accuracy'],
            'Kappa': metrics['kappa_coefficient']
        })
        
print("-" * 60)
print("Analysis complete! View in MLflow UI: mlflow ui --port 5000")

In [ ]:
# Visualize results
import pandas as pd

df = pd.DataFrame(results)
print("\nσ Sensitivity Results:")
print(df.to_string(index=False))

# Find best sigma
best_idx = df['OA'].idxmax()
best_sigma = df.loc[best_idx, 'sigma']
best_oa = df.loc[best_idx, 'OA']
print(f"\n★ Best σ = {best_sigma} with OA = {best_oa:.4f}")

In [ ]:
# Plot sensitivity curve
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

metrics_to_plot = ['OA', 'AA', 'Kappa']
titles = ['Overall Accuracy', 'Average Accuracy', 'Kappa Coefficient']

for ax, metric, title in zip(axes, metrics_to_plot, titles):
    ax.semilogx(df['sigma'], df[metric], 'o-', linewidth=2, markersize=8)
    ax.axvline(x=best_sigma, color='r', linestyle='--', alpha=0.7, label=f'Best σ={best_sigma}')
    ax.set_xlabel('σ (log scale)')
    ax.set_ylabel(title)
    ax.set_title(f'{title} vs σ')
    ax.grid(True, alpha=0.3)
    ax.legend()

plt.suptitle('RBF Kernel σ Sensitivity Analysis', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig('sigma_sensitivity_curve.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Edge Case Analysis

Analyze how σ affects classification of "mixed pixels" near class boundaries.

In [ ]:
# Find edge pixels (pixels adjacent to different classes)
from scipy.ndimage import sobel

# Compute gradient magnitude on ground truth to find edges
edge_x = sobel(ground_truth.astype(float), axis=0)
edge_y = sobel(ground_truth.astype(float), axis=1)
edge_magnitude = np.sqrt(edge_x**2 + edge_y**2)

# Edge pixels are where gradient is non-zero and there's a label
edge_mask = (edge_magnitude > 0) & (ground_truth > 0)
interior_mask = (edge_magnitude == 0) & (ground_truth > 0)

print(f"Edge pixels: {np.sum(edge_mask)}")
print(f"Interior pixels: {np.sum(interior_mask)}")

In [ ]:
# Analyze edge vs interior accuracy for different sigma
# Get test set positions
test_positions = test_set.positions

edge_acc_per_sigma = []
interior_acc_per_sigma = []

for sigma in tqdm(sigma_test_values, desc="Edge analysis"):
    gamma = 1.0 / (2 * sigma ** 2)
    
    svm = SVMClassifier(kernel='rbf', C=10.0, gamma=gamma)
    svm.fit(X_train, y_train, log_to_mlflow=False)
    y_pred = svm.predict(X_test)
    
    # Classify test samples as edge or interior
    test_edge_mask = np.array([edge_mask[r, c] for r, c in test_positions])
    test_interior_mask = np.array([interior_mask[r, c] for r, c in test_positions])
    
    # Compute accuracy for each group
    if np.sum(test_edge_mask) > 0:
        edge_acc = np.mean(y_pred[test_edge_mask] == y_test[test_edge_mask])
    else:
        edge_acc = 0
    
    if np.sum(test_interior_mask) > 0:
        interior_acc = np.mean(y_pred[test_interior_mask] == y_test[test_interior_mask])
    else:
        interior_acc = 0
    
    edge_acc_per_sigma.append(edge_acc)
    interior_acc_per_sigma.append(interior_acc)

print("\nEdge vs Interior Accuracy:")
for sigma, edge_acc, int_acc in zip(sigma_test_values, edge_acc_per_sigma, interior_acc_per_sigma):
    print(f"σ={sigma:6.3f}  Edge: {edge_acc:.4f}  Interior: {int_acc:.4f}  Gap: {int_acc - edge_acc:.4f}")

In [ ]:
# Plot edge vs interior accuracy
fig, ax = plt.subplots(figsize=(10, 6))

ax.semilogx(sigma_test_values, interior_acc_per_sigma, 'g-o', label='Interior Pixels', linewidth=2, markersize=8)
ax.semilogx(sigma_test_values, edge_acc_per_sigma, 'r-s', label='Edge Pixels', linewidth=2, markersize=8)
ax.fill_between(sigma_test_values, edge_acc_per_sigma, interior_acc_per_sigma, alpha=0.2, color='gray')

ax.axvline(x=best_sigma, color='blue', linestyle='--', alpha=0.7, label=f'Best σ={best_sigma}')

ax.set_xlabel('σ (log scale)', fontsize=12)
ax.set_ylabel('Accuracy', fontsize=12)
ax.set_title('Edge vs Interior Pixel Classification Accuracy', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

plt.savefig('edge_interior_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n★ Key Finding: Edge pixels are harder to classify. The gap between")
print("  interior and edge accuracy varies with σ - this is important for thesis!")

## 7. Conclusions

### Key Findings

1. **Optimal σ range**: Based on the analysis, the best σ for this data is around X
2. **Edge pixel sensitivity**: σ significantly affects edge pixel accuracy
3. **Trade-off**: Small σ → local precision, Large σ → global smoothness

### Thesis Implications

- Document the optimal σ range for different datasets
- Use edge pixel accuracy as a key metric for local similarity evaluation
- Consider adaptive σ based on local spectral variance

In [ ]:
# Save summary for thesis
summary = f"""
========================================
σ SENSITIVITY ANALYSIS SUMMARY
========================================

Dataset: {'Indian Pines' if USE_REAL_DATA else 'Synthetic'}
Patch size: {patch_size}x{patch_size}
Training samples: {len(X_train)}
Test samples: {len(X_test)}

Best σ: {best_sigma}
Best Overall Accuracy: {best_oa:.4f}

Edge Pixel Analysis:
- Edge pixels: {np.sum(test_edge_mask)} ({100*np.sum(test_edge_mask)/len(y_test):.1f}% of test)
- Interior pixels: {np.sum(test_interior_mask)} ({100*np.sum(test_interior_mask)/len(y_test):.1f}% of test)

σ Sensitivity Table:
{df.to_string(index=False)}
"""

print(summary)

# Save to file
with open('sigma_analysis_summary.txt', 'w') as f:
    f.write(summary)
print("\n✓ Summary saved to sigma_analysis_summary.txt")